## Utils

### Configuration

In [33]:
from pathlib import Path
from typing import Union, Dict, Any


class Config:
    # Embedded configuration data
    _config_data: Dict[str, Any] = {
        "log_level": "DEBUG",
        "tucson_center_coordinates": [32.2226, -110.9747],
        "tucson_bounds": [-111, 32.0, -110.9747, 32.2226],
        "paths": {"visualizations": "visualizations"},
        "data_sources": {
            "arrests": "https://raw.githubusercontent.com/christian-byrne/tucson-crime-models/main/data/crime/Tucson_Police_Arrests_-_2017_-_Open_Data.geojson",
            "sidewalks": "https://raw.githubusercontent.com/christian-byrne/tucson-crime-models/main/data/infrastructure/Sidewalks.geojson",
        },
    }

    # Root path for resolving paths
    ROOT_PATH: Path = Path(".").resolve()

    @staticmethod
    def get_proj_root(cls) -> Path:
        """Return the root path of the project."""
        return Config.ROOT_PATH

    def __getitem__(self, key: str) -> Any:
        """Get a configuration value by key, with KeyError if missing."""
        if key not in Config._config_data:
            msg = (
                f"Could not find key '{key}' in config."
                + f"Available keys: {list(Config._config_data.keys())}"
            )
            raise KeyError(msg)
        return Config._config_data[key]

    def __setitem__(self, key: str, value: Union[str, Path]) -> None:
        """Set a configuration value by key, converting Path to string if necessary."""
        if isinstance(value, Path):
            value = str(value.resolve())
        Config._config_data[key] = value

    @classmethod
    def get_all(cls) -> Dict[str, Any]:
        """Return the entire configuration data dictionary."""
        return cls._config_data
    
config = Config()

### Logging

In [34]:
import logging
from rich.logging import RichHandler


class Logger:
    def __init__(self, logger_name: str, level: int = logging.INFO):
        self.logger_base = logging.getLogger(logger_name)

        # Only add the RichHandler if there are no existing handlers
        if not self.logger_base.handlers:
            rich_handler = RichHandler(
                rich_tracebacks=True, tracebacks_show_locals=True, markup=True
            )

            formatter = logging.Formatter("%(message)s")
            rich_handler.setFormatter(formatter)

            self.logger_base.addHandler(rich_handler)

        # Set the logging level
        self.logger_base.setLevel(level)

    def __call__(self):
        return self.logger_base


logger = Logger("General", config["log_level"])()

### File Loading

In [35]:
import geopandas as gpd
import requests
from typing import Dict

# Dictionary to cache datasets in memory to prevent re-fetching in same session
_cache: Dict[str, gpd.GeoDataFrame] = {}


def load_dataset(dataset_name: str):
    # Check if the dataset is already cached
    if dataset_name in _cache:
        logger.info(f"Using cached data for {dataset_name}")
        return _cache[dataset_name]

    # Load the dataset from Config if not cached
    filepath = config["data_sources"][dataset_name]

    if filepath.startswith("http"):
        logger.info(f"Downloading dataset from {filepath}")
        response = requests.get(filepath)
        response.raise_for_status()
        with open(f"/tmp/{dataset_name}.geojson", "wb") as f:
            f.write(response.content)
        dataframe = gpd.read_file(f"/tmp/{dataset_name}.geojson")
    else:
        dataframe = gpd.read_file(filepath)

    logger.info(f"Loaded data from {filepath}")
    logger.debug(
        f"Columns in {filepath.split('/')[-1]}:\n{', '.join(dataframe.columns)}"
    )

    # Cache the loaded dataset
    _cache[dataset_name] = dataframe

    return dataframe

### Exploratory Data Analysis

#### Data Cleaning

##### Filter Geographic Information System (GIS) Data by Bounds

In [36]:
import folium
import geopandas as gpd
from shapely.geometry import box


def filter_by_bounds(data: gpd.GeoDataFrame, bounds: tuple) -> gpd.GeoDataFrame:
    """
    Filters the GeoDataFrame to include only points within the specified bounds.

    Parameters:
    data (GeoDataFrame): The data to filter.
    bounds (tuple): A tuple (minx, miny, maxx, maxy) defining the bounding box.

    Returns:
    GeoDataFrame: Filtered GeoDataFrame with points within the bounds.
    """
    logger.debug(f"Filtering data by bounds: {bounds}")
    bbox = box(*bounds)

    if len(data) == 0:
        logger.error("No data found in the input GeoDataFrame. Check the input data.")
        return data

    # Filter data by checking if geometry is within the bounding box
    before_length = len(data)
    filtered = data[data.geometry.within(bbox)]
    logger.info(f"Filtered out {before_length - len(filtered)} rows outside the bounds")

    if len(data) == len(filtered):
        logger.warning("All data is within the specified bounds. No data was filtered.")

    if len(filtered) == 0:
        logger.error(
            f"All data was filtered out. Check that the bounds {bounds} are correct."
        )

    return filtered

#### Data Visualization

##### Plot GIS Data on a Geographic Map

In [37]:
from typing import Optional


def create_map(
    data: gpd.GeoDataFrame,
    location: list = None,
    zoom_start: int = 12,
    popup_field: Optional[str] = None,
    save_path: Union[str, Path, None] = None,
):
    """
    Create a folium map with markers for each point in the input GeoDataFrame.

    Parameters:
    data (GeoDataFrame): The data to plot, must contain 'geometry' column.
    location (list): Latitude and longitude to center the map on. Default is Tucson, AZ.
    zoom_start (int): Initial zoom level for the map.
    popup_field (str, optional): Column name to use for the marker popups. If None, no popups.
    save_path (str or Path, optional): Path to save the generated map HTML file.

    Returns:
    folium.Map: A map with a marker for each point in data.
    """
    # Default location to Tucson, AZ if not provided
    location = location or [32.2226, -110.9747]

    logger.info(f"Creating map centered at {location} with zoom level {zoom_start}...")
    crime_map = folium.Map(location=location, zoom_start=zoom_start)

    logger.debug("Filtering out rows with missing or non-Point geometry")
    data = data[data.geometry.notnull() & (data.geometry.type == "Point")]

    logger.info(f"Adding {len(data)} points to the map")
    for _, row in data.iterrows():
        # Retrieve the popup content if specified and available
        popup_text = (
            str(row[popup_field]) if popup_field and popup_field in row else None
        )
        folium.Marker([row.geometry.y, row.geometry.x], popup=popup_text).add_to(
            crime_map
        )

    # Save map to HTML if save_path is specified
    if save_path:
        save_path = Path(save_path) if isinstance(save_path, str) else save_path
        crime_map.save(save_path)
        logger.info(f"Map saved at {save_path}")

    # Display the map inline in the notebook
    # display(crime_map)

    return crime_map

#### Data Joining

In [38]:
def join_data(
    left: gpd.GeoDataFrame,
    right: gpd.GeoDataFrame,
    left_key: str,
    right_key: str,
    how: str = "left",
) -> gpd.GeoDataFrame:
    """
    Join two GeoDataFrames based on a common key.

    Parameters:
    left (GeoDataFrame): The left GeoDataFrame to join.
    right (GeoDataFrame): The right GeoDataFrame to join.
    left_key (str): The column name to join on in the left GeoDataFrame.
    right_key (str): The column name to join on in the right GeoDataFrame.
    how (str): The type of join to perform. Default is 'left'.

    Returns:
    GeoDataFrame: A GeoDataFrame with the joined data.
    """
    logger.info(f"Joining data on {left_key} with {right_key} using {how} join")
    joined = left.merge(right, how=how, left_on=left_key, right_on=right_key)

    if len(joined) == 0:
        logger.error(
            f"No data was joined. Check that the keys {left_key} and {right_key} are correct."
        )

    return joined

##### Join GIS Data based on Distance Comparisons

In [39]:
# NOTE: This approach works well for moderately sized datasets but could become slow for very large ones. For large datasets, spatial indexing or more advanced spatial joins might be needed (e.g., using scipy’s cKDTree for faster nearest-neighbor searches).
def distance_join(
    main_gdf, ref_gdf, ref_col_name="nearest_id", distance_col_name="distance_to"
):
    """
    Adds a column to the main GeoDataFrame with the ID of the nearest feature
    from the reference GeoDataFrame.

    Parameters:
    main_gdf (GeoDataFrame): The main GeoDataFrame, typically with Point geometries (e.g., arrests data).
    ref_gdf (GeoDataFrame): The reference GeoDataFrame to find the nearest feature from (e.g., sidewalks).
    ref_col (str): The name of the new column in main_gdf to store the ID of the nearest feature.
    distance_col (str, optional): If provided, stores the distance to the nearest feature in this column.

    Returns:
    GeoDataFrame: The main GeoDataFrame with an added column for the nearest feature's ID.
    """
    # Ensure both GeoDataFrames are in the same CRS, convert to a metric CRS for distance calculation
    metric_crs = "EPSG:3857"
    main_gdf = main_gdf.to_crs(metric_crs)
    ref_gdf = ref_gdf.to_crs(metric_crs)

    # Prepare the new columns in main_gdf
    main_gdf[ref_col_name] = None
    if distance_col_name:
        main_gdf[distance_col_name] = None

    # For each geometry in main_gdf, find the closest geometry in ref_gdf
    for idx, main_geom in main_gdf.iterrows():
        # Find the nearest geometry in ref_gdf
        nearest_geom = ref_gdf.geometry.distance(main_geom.geometry).idxmin()

        # Add the ID of the nearest geometry (index) to the main_gdf
        main_gdf.at[idx, ref_col_name] = ref_gdf.at[
            nearest_geom, "OBJECTID"
        ]  # or another ID column in ref_gdf

        # Optionally, store the distance to the nearest feature
        if distance_col_name:
            distance = main_geom.geometry.distance(ref_gdf.at[nearest_geom, "geometry"])
            main_gdf.at[idx, distance_col_name] = distance

    # Reproject main_gdf back to the original CRS
    main_gdf = main_gdf.to_crs(ref_gdf.crs)

    return main_gdf

## Main

#### Load *Arrests* Data

In [ ]:
arrests_data = load_dataset("arrests")

#### Clean Data

In [41]:
arrests_data = arrests_data[arrests_data.geometry.notnull()]

#### Truncate/Shrink Data for Fast Testing (Temporary)

In [ ]:
# Restrict to just Tucson
arrests_data = filter_by_bounds(arrests_data, config["tucson_bounds"])

# Truncate
arrests_data = arrests_data.head(300)

#### Visualize Arrests Data

In [ ]:
create_map(arrests_data, popup_field="ArrestChargeandDescription")

#### Load *Sidewalks* Data

In [ ]:
sidewalks_data = load_dataset("sidewalks")

#### Add `distance_to_nearest_sidewalk` to Arrests Data

In [45]:
arrests_data = distance_join(
    arrests_data,
    sidewalks_data,
    ref_col_name="nearest_sidewalk_id",
    distance_col_name="distance_to_nearest_sidewalk",
)

#### Analyze Correlation between Arrests Frequency and `distance_to_nearest_sidewalk`

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.histplot(arrests_data["distance_to_nearest_sidewalk"], kde=True, bins=30)
plt.xlabel("Distance to Nearest Sidewalk (meters)")
plt.ylabel("Frequency")
plt.title("Distribution of Distance to Nearest Sidewalk for Crime Incidents")
plt.show()